In [31]:
import numpy as np
import pandas_ta as ta
import pandas as pd
from datetime import datetime, timedelta, timezone
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

In [35]:
df_min = pd.read_csv('vn30f1m_3min.csv')
df_hour = pd.read_csv('vn30f1m_1hour.csv')
df_day = pd.read_csv('vn30f1m_1day.csv')

In [36]:
def data_backtest(data):
    new_data = data[['Open', 'High', 'Low', 'Close', 'Volume']]
    new_data.index = pd.to_datetime(data['Date'])
    new_data.index.name = None
    return new_data

In [37]:
def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

In [48]:
def process_data(df):
    data = df.copy()
    data = data[~data.index.duplicated(keep='first')] # Handling duplicate
    
    data['Date'] = [str(i)[:10] for i in data.index]
    data['time'] = [str(i)[11:] for i in data.index]
    data_model = data.pivot(index = 'Date', columns = 'time', values = ['Open','High','Low','Close','Volume']).ffill(axis = 1).stack().reset_index() # Handling missing values

    return data_model

In [38]:
data_min = data_backtest(df_min)
data_hour = data_backtest(df_hour)
data_day = data_backtest(df_day)

In [39]:
train_min, valid_min = split_data(data_min)
train_hour, valid_hour = split_data(data_hour)
train_day, valid_day = split_data(data_day)

### Mẫu

    def indicator(data):
        Hàm indicator
        formular

        return np.array(something)

    class CustomStrategy(Strategy):

        def init(self):
            Note that this returns each column as a row
            Then extends the length of each row on each iteration
            self.something = self.I(indicator, self.data)

        def next(self):

            sth1 = self.something[0]
            sth2 = self.something[2]

            if self.position:
                if something happens:
                    self.position.close()
            else:
                if something happens:
                    self.buy(size=0.7)
                
    bt = Backtest(data, CustomStrategy, cash=3000, commission=0.01, margin=0.13, hedging=True, exclusive_orders=False)
    stats = bt.run()
    stats

# Momentum & Volatility

In [66]:
# Hàm tính toán Awesome Oscillator (AO)
def awesome_oscillator(data, window1, window2):
    high = data['High']
    low = data['Low']
    median_price = (high + low) / 2

    # Tính SMA cho 5 và 34 kỳ
    sma_5 = pd.Series(median_price).rolling(window=window1).mean()
    sma_34 = pd.Series(median_price).rolling(window=window2).mean()

    # AO = SMA(5) - SMA(34)
    ao = sma_5 - sma_34
    return np.array(ao)

# Hàm tính toán Average True Range (ATR)
def average_true_range(data, period=14):
    high = pd.Series(data['High'])
    low = pd.Series(data['Low'])
    close = pd.Series(data['Close'])

    # True Range = max(High - Low, High - Previous Close, Previous Close - Low)
    high_low = high - low
    high_close = (high - close.shift(1)).abs()
    low_close = (low - close.shift(1)).abs()

    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    # ATR = SMA của True Range
    atr = true_range.rolling(window=period).mean()
    return atr

class AO_ATV_Strategy(Strategy):
    window1 = 5
    window2 = 34
    period = 14


    def init(self):
        # Lưu trữ chỉ báo AO và ATR
        self.ao = self.I(awesome_oscillator, self.data, self.window1, self.window2)
        self.atr = self.I(average_true_range, self.data, self.period)

    def next(self):
        ao = self.ao[-1]  # Lấy giá trị gần nhất của AO
        atr = self.atr[-1]  # Lấy giá trị gần nhất của ATR

        # Điều kiện giao dịch: AO > 0 và ATR lớn hơn một mức ngưỡng (ví dụ: 0.5)
        if self.position:
            if ao < 0:
                self.position.close()
        else:
            if ao > 0 and atr > 0.7:
                self.buy(size=0.7)

# Giả định rằng `data` là DataFrame chứa dữ liệu giá với các cột 'Open', 'High', 'Low', 'Close', 'Volume'
bt = Backtest(train_day, AO_ATV_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True, exclusive_orders=False)
stats = bt.run()
print(stats)

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   56.231306
Equity Final [$]                      1609.94
Equity Peak [$]                      4002.255
Return [%]                         -46.335333
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                  -14.611124
Volatility (Ann.) [%]               68.059622
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -81.576933
Avg. Drawdown [%]                  -24.531539
Max. Drawdown Duration      963 days 00:00:00
Avg. Drawdown Duration      203 days 00:00:00
# Trades                                   19
Win Rate [%]                        36.842105
Best Trade [%]                      35.060633
Worst Trade [%]                     -8.925541
Avg. Trade [%]                    

In [67]:
stats = bt.optimize(
    window1 = range(3, 9, 1),
    window2 = range(30, 40, 2),
    period = range(10, 20, 2),
    maximize = "Sharpe Ratio",
    #constraint = lambda param: param.window2 > param.window1
)

In [68]:
print(stats)

Start                     2018-08-13 07:00:00
End                       2022-08-30 09:00:00
Duration                   1478 days 02:00:00
Exposure Time [%]                   59.222333
Equity Final [$]                      581.312
Equity Peak [$]                      3619.566
Return [%]                         -80.622933
Buy & Hold Return [%]               34.877384
Return (Ann.) [%]                    -34.0628
Volatility (Ann.) [%]               54.446911
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -86.304656
Avg. Drawdown [%]                  -30.718174
Max. Drawdown Duration      971 days 00:00:00
Avg. Drawdown Duration      239 days 00:00:00
# Trades                                   26
Win Rate [%]                        38.461538
Best Trade [%]                      21.335786
Worst Trade [%]                     -9.372532
Avg. Trade [%]                    

In [69]:
# bt.plot()

In [84]:
# Hàm tính Donchian Channel, trả về dạng numpy array
def donchian_channel(data, period=20):
    high = pd.Series(data['High'])
    low = pd.Series(data['Low'])

    upper_band = high.rolling(window=period).max().to_numpy()
    lower_band = low.rolling(window=period).min().to_numpy()
    mid_band = ((upper_band + lower_band) / 2)

    return upper_band, lower_band, mid_band

# Hàm tính Kaufman's Adaptive Moving Average (KAMA), trả về dạng numpy array
def kaufman_adaptive_moving_average(data, period=10, fast_period=2, slow_period=30):
    close = pd.Series(data['Close'])
    
    change = close.diff(period)
    volatility = close.diff(1).abs().rolling(window=period).sum()

    efficiency_ratio = (change.abs() / volatility).to_numpy()
    smoothing_constant = (efficiency_ratio * (2 / (fast_period + 1) - 2 / (slow_period + 1)) + 2 / (slow_period + 1)) ** 2

    kama = [close.iloc[0]]  # Khởi tạo KAMA với giá trị đầu tiên
    for i in range(1, len(close)):
        kama.append(kama[-1] + smoothing_constant[i] * (close.iloc[i] - kama[-1]))

    return np.array(kama)

class DC_KAMA_Strategy(Strategy):

    def init(self):
        # Lưu trữ chỉ báo Donchian Channel dưới dạng numpy array
        self.upper_band, self.lower_band, self.mid_band = self.I(donchian_channel, self.data)
        # Lưu trữ KAMA dưới dạng numpy array
        self.kama = self.I(kaufman_adaptive_moving_average, self.data)

    def next(self):
        upper_band = self.upper_band[-1]  # Giá trị gần nhất của dải trên (numpy array)
        lower_band = self.lower_band[-1]  # Giá trị gần nhất của dải dưới (numpy array)
        kama = self.kama[-1]  # Giá trị gần nhất của KAMA (numpy array)

        # Điều kiện giao dịch đơn giản hơn
        if self.position:
            if self.data.Close[-1] > 0.95*upper_band:  # Đóng vị thế nếu giá vượt dải trên
                self.position.close()
        else:
            if self.data.Close[-1] > 0.95*lower_band:  # Mua nếu giá vượt dải dưới
                self.buy(size=0.7)
                self.buy(size=0.7)

# Giả định rằng `data` là DataFrame chứa dữ liệu giá với các cột 'Open', 'High', 'Low', 'Close', 'Volume'
bt = Backtest(train_hour, DC_KAMA_Strategy, cash=3000, commission=0.01, margin=0.13, hedging=True, exclusive_orders=False)
stats = bt.run()
print(stats)

Start                     2018-08-13 09:00:00
End                       2022-08-25 11:00:00
Duration                   1473 days 02:00:00
Exposure Time [%]                    3.201432
Equity Final [$]                       47.763
Equity Peak [$]                        3000.0
Return [%]                           -98.4079
Buy & Hold Return [%]               36.585624
Return (Ann.) [%]                  -64.514886
Volatility (Ann.) [%]               35.236498
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -99.441233
Avg. Drawdown [%]                  -99.441233
Max. Drawdown Duration     1469 days 02:00:00
Avg. Drawdown Duration     1469 days 02:00:00
# Trades                                   88
Win Rate [%]                         1.136364
Best Trade [%]                       1.643265
Worst Trade [%]                    -12.965298
Avg. Trade [%]                    